In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

In [2]:
#Loading Dataset

# --- Online Payment Fraud ---
online_payment_data = pd.read_csv('onlinefraud.csv')

# --- Credit Card Fraud ---
credit_card_data = pd.read_csv('creditcard.csv')


In [5]:
#Preprocessing

# --- Online Payment Fraud ---
type_mapping = {'CASH_OUT':0, 'PAYMENT':1, 'CASH_IN':2, 'TRANSFER':3, 'DEBIT':4}
online_payment_data['type_encoded'] = online_payment_data['type'].map(type_mapping)

# New Features
online_payment_data['errorBalanceOrig'] = online_payment_data['oldbalanceOrg'] - online_payment_data['amount'] - online_payment_data['newbalanceOrig']
online_payment_data['errorBalanceDest'] = online_payment_data['newbalanceDest'] + online_payment_data['amount'] - online_payment_data['oldbalanceDest']

features_online = ['type_encoded', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'errorBalanceOrig', 'errorBalanceDest']
X_online = online_payment_data[features_online]
y_online = online_payment_data['isFraud']

# --- Credit Card Fraud ---
features_credit = credit_card_data.drop(columns=['Time', 'Class'])
labels_credit = credit_card_data['Class']

# Remove NaN rows if any
features_credit = features_credit.dropna()
labels_credit = labels_credit.loc[features_credit.index]

# Balance dataset
smote = SMOTE(random_state=42, k_neighbors=1)
X_credit, y_credit = smote.fit_resample(features_credit, labels_credit)

In [7]:
# Drop NaNs
data_online = pd.concat([X_online, y_online], axis=1)
data_online = data_online.dropna()

X_online = data_online[features_online]
y_online = data_online['isFraud']


In [8]:
# Train-Test Splitting

X_train_online, X_test_online, y_train_online, y_test_online = train_test_split(X_online, y_online, test_size=0.2, stratify=y_online, random_state=42)
X_train_credit, X_test_credit, y_train_credit, y_test_credit = train_test_split(X_credit, y_credit, test_size=0.2, random_state=42)

In [9]:
#Training Models

# --- Online Payment Fraud Model ---
model_online = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
model_online.fit(X_train_online, y_train_online)

# --- Credit Card Fraud Model ---
model_credit = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
model_credit.fit(X_train_credit, y_train_credit)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [10]:
# Evaluating Models

# Online Payment
print("\n===== Online Payment Fraud Detection Report =====\n")
y_pred_online = model_online.predict(X_test_online)
print(classification_report(y_test_online, y_pred_online))
print(confusion_matrix(y_test_online, y_pred_online))

# Credit Card
print("\n===== Credit Card Fraud Detection Report =====\n")
y_pred_credit = model_credit.predict(X_test_credit)
print(classification_report(y_test_credit, y_pred_credit))
print(confusion_matrix(y_test_credit, y_pred_credit))


===== Online Payment Fraud Detection Report =====

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5643
         1.0       1.00      1.00      1.00        17

    accuracy                           1.00      5660
   macro avg       1.00      1.00      1.00      5660
weighted avg       1.00      1.00      1.00      5660

[[5643    0]
 [   0   17]]

===== Credit Card Fraud Detection Report =====

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       815
         1.0       1.00      1.00      1.00       773

    accuracy                           1.00      1588
   macro avg       1.00      1.00      1.00      1588
weighted avg       1.00      1.00      1.00      1588

[[815   0]
 [  0 773]]


In [11]:
# Saving Models

with open('online_payment_model.sav', 'wb') as f:
    pickle.dump(model_online, f)

with open('credit_card_model.sav', 'wb') as f:
    pickle.dump(model_credit, f)

print("\n✅ Models Saved Successfully!")



✅ Models Saved Successfully!
